In [10]:
import pandas as pd
import numpy as np

INPUT_FILE = "../data/transactions.csv"
OUTPUT_FILE = "../data/transactions_features.csv"

df = pd.read_csv(INPUT_FILE, parse_dates=["datetime"])
df["hour"] = df["datetime"].dt.hour
df["day_of_week"] = df["datetime"].dt.weekday  # Monday=0
df["is_weekend"] = df["day_of_week"] >= 5

def time_bucket(hour):
    if 6 <= hour < 12:
        return "MORNING"
    elif 12 <= hour < 18:
        return "AFTERNOON"
    elif 18 <= hour < 22:
        return "EVENING"
    else:
        return "LATE_NIGHT"

df["time_bucket"] = df["hour"].apply(time_bucket)
df.head()

,transaction_id,user_id,datetime,amount,merchant,category,channel,payment_type,location,hour,day_of_week,is_weekend,time_bucket
0,TXN000003,U001,2025-01-01 12:20:00,372.27,LAZADA,Shopping,E-WALLET,DEBIT,Kuala Lumpur,12,2,False,AFTERNOON
1,TXN000001,U001,2025-01-01 14:11:00,357.83,UNIQLO,Shopping,E-WALLET,DEBIT,Petaling Jaya,14,2,False,AFTERNOON
2,TXN000002,U001,2025-01-01 15:41:00,83.12,SETEL,Fuel,DEBIT CARD,DEBIT,Petaling Jaya,15,2,False,AFTERNOON
3,TXN000004,U001,2025-01-01 18:03:00,423.52,SHOPEE,Shopping,DEBIT CARD,DEBIT,Petaling Jaya,18,2,False,EVENING
4,TXN000008,U001,2025-01-02 12:49:00,185.36,LOTUS,Grocery,E-WALLET,DEBIT,Shah Alam,12,3,False,AFTERNOON


In [11]:

category_stats = (
    df.groupby("category")["amount"]
    .agg(["mean", "std", "count"])
    .reset_index()
    .rename(columns={
        "mean": "category_avg_amount",
        "std": "category_std_amount",
        "count": "category_txn_count"
    })
)

df = df.merge(category_stats, on="category", how="left")
df["category_std_amount"] = df["category_std_amount"].fillna(1)
df.head()

,transaction_id,user_id,datetime,amount,merchant,category,channel,payment_type,location,hour,day_of_week,is_weekend,time_bucket,category_avg_amount,category_std_amount,category_txn_count
0,TXN000003,U001,2025-01-01 12:20:00,372.27,LAZADA,Shopping,E-WALLET,DEBIT,Kuala Lumpur,12,2,False,AFTERNOON,287.100621,137.937087,177
1,TXN000001,U001,2025-01-01 14:11:00,357.83,UNIQLO,Shopping,E-WALLET,DEBIT,Petaling Jaya,14,2,False,AFTERNOON,287.100621,137.937087,177
2,TXN000002,U001,2025-01-01 15:41:00,83.12,SETEL,Fuel,DEBIT CARD,DEBIT,Petaling Jaya,15,2,False,AFTERNOON,83.209789,22.891279,142
3,TXN000004,U001,2025-01-01 18:03:00,423.52,SHOPEE,Shopping,DEBIT CARD,DEBIT,Petaling Jaya,18,2,False,EVENING,287.100621,137.937087,177
4,TXN000008,U001,2025-01-02 12:49:00,185.36,LOTUS,Grocery,E-WALLET,DEBIT,Shah Alam,12,3,False,AFTERNOON,141.853503,61.778727,177


In [12]:
merchant_stats = (
    df.groupby("merchant")["amount"]
    .agg(["mean", "count"])
    .reset_index()
    .rename(columns={
        "mean": "merchant_avg_amount",
        "count": "merchant_txn_count"
    })
)

df = df.merge(merchant_stats, on="merchant", how="left")
df.head()


,transaction_id,user_id,datetime,amount,merchant,category,channel,payment_type,location,hour,day_of_week,is_weekend,time_bucket,category_avg_amount,category_std_amount,category_txn_count,merchant_avg_amount,merchant_txn_count
0,TXN000003,U001,2025-01-01 12:20:00,372.27,LAZADA,Shopping,E-WALLET,DEBIT,Kuala Lumpur,12,2,False,AFTERNOON,287.100621,137.937087,177,299.896275,51
1,TXN000001,U001,2025-01-01 14:11:00,357.83,UNIQLO,Shopping,E-WALLET,DEBIT,Petaling Jaya,14,2,False,AFTERNOON,287.100621,137.937087,177,296.040000,35
2,TXN000002,U001,2025-01-01 15:41:00,83.12,SETEL,Fuel,DEBIT CARD,DEBIT,Petaling Jaya,15,2,False,AFTERNOON,83.209789,22.891279,142,85.284762,42
3,TXN000004,U001,2025-01-01 18:03:00,423.52,SHOPEE,Shopping,DEBIT CARD,DEBIT,Petaling Jaya,18,2,False,EVENING,287.100621,137.937087,177,252.187333,45
4,TXN000008,U001,2025-01-02 12:49:00,185.36,LOTUS,Grocery,E-WALLET,DEBIT,Shah Alam,12,3,False,AFTERNOON,141.853503,61.778727,177,149.890000,47


In [13]:
df["amount_zscore"] = (
    (df["amount"] - df["category_avg_amount"]) /
    df["category_std_amount"]
)

df["category_avg_amount"] = df["category_avg_amount"].round(2)
df["category_std_amount"] = df["category_std_amount"].round(2)
df["merchant_avg_amount"] = df["merchant_avg_amount"].round(2)
df["amount_zscore"] = df["amount_zscore"].round(2)
df.head()

,transaction_id,user_id,datetime,amount,merchant,category,channel,payment_type,location,hour,day_of_week,is_weekend,time_bucket,category_avg_amount,category_std_amount,category_txn_count,merchant_avg_amount,merchant_txn_count,amount_zscore
0,TXN000003,U001,2025-01-01 12:20:00,372.27,LAZADA,Shopping,E-WALLET,DEBIT,Kuala Lumpur,12,2,False,AFTERNOON,287.10,137.94,177,299.90,51,0.62
1,TXN000001,U001,2025-01-01 14:11:00,357.83,UNIQLO,Shopping,E-WALLET,DEBIT,Petaling Jaya,14,2,False,AFTERNOON,287.10,137.94,177,296.04,35,0.51
2,TXN000002,U001,2025-01-01 15:41:00,83.12,SETEL,Fuel,DEBIT CARD,DEBIT,Petaling Jaya,15,2,False,AFTERNOON,83.21,22.89,142,85.28,42,-0.00
3,TXN000004,U001,2025-01-01 18:03:00,423.52,SHOPEE,Shopping,DEBIT CARD,DEBIT,Petaling Jaya,18,2,False,EVENING,287.10,137.94,177,252.19,45,0.99
4,TXN000008,U001,2025-01-02 12:49:00,185.36,LOTUS,Grocery,E-WALLET,DEBIT,Shah Alam,12,3,False,AFTERNOON,141.85,61.78,177,149.89,47,0.70


In [14]:
df = df.sort_values("datetime")
df.to_csv(OUTPUT_FILE, index=False)

print(f"Feature engineering completed → {OUTPUT_FILE}")

Feature engineering completed → ../data/transactions_features.csv
